### purpose

fit trained GF models from multivariate sims that used nuisance envs to the common gardens of the environment

this code is inspired by 02_analysis/07_experiments/02_nuisance_envs/02_fit_and_validate_GF_nuis_envs.ipynb

In [1]:
from pythonimports import *

import MVP_summary_functions as mvp
import MVP_02_fit_gradient_forests as mvp02

nuis_dir = '/home/b.lind/offsets/run_20220919_nuisance/tutorial'

gf_training_dir = f'{nuis_dir}/gradient_forests/training/training_files'
gf_shdir = gf_training_dir.replace('_files', '_shfiles')
gf_outdir = gf_shdir.replace('_shfiles', '_outfiles')

t0 = dt.now()

mvp.latest_commit()
session_info.show()

#########################################################
Today:	June 02, 2023 - 11:22:36
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit c721bd0d89c1e150e17a4e0739c26e4296405233  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Wed May 10 15:13:03 2023 -0400

Current commit of MVP_offsets:
commit 060db40dff8355d2087deab28bd695fcccee8f49  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Thu May 18 12:50:56 2023 -0400
#########################################################



# if __name__ == '__main__'

### input args

In [2]:
mvp02.seed = 'tutorial'

In [3]:
mvp02.rscript_exe = '/home/b.lind/anaconda3/envs/r35/lib/R/bin/Rscript'

In [4]:
mvp02.training_outdir = gf_outdir

### start cluster

In [5]:
lview, dview = get_client(cluster_id='1685714034-x2t5', profile='lotterhos')

36 36


### load objects to clusterm

In [6]:
dview['rscript_exe'] = mvp02.rscript_exe
mvp02.fitting_file = '/home/b.lind/code/MVP-offsets/01_src/MVP_gf_fitting_script.R'
dview['fitting_file'] = mvp02.fitting_file

mvp02.lview = lview  # for mvp02.run_fit_gradient_forests

### create dirs

In [7]:
mvp02.training_outdir

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_outfiles'

In [8]:
garden_dir, mvp02.fitting_dir, training_filedir = mvp02.make_fitting_dirs(mvp02.training_outdir)

mvp02.garden_dir = garden_dir


Creating directories ...
	gf_directory = '/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests'
	fitting_dir = '/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/fitting/fitting_outfiles'
	garden_dir = '/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/fitting/garden_files'
	training_filedir = '/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_files'


# main

### get environmental data

In [9]:
# get_envdata (skip part of function that asserts two envfiles, one for ind and one for pooled)
files = fs(training_filedir, pattern='_envfile_')

envdfs = {}
envfiles = {}
for f in files:
    ind_or_pooled = op.basename(f).split("_")[-1].rstrip(".txt")
    envdfs[ind_or_pooled] = pd.read_table(f, index_col=0)
    envfiles[ind_or_pooled] = f

In [10]:
envfiles

{'pooled': '/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_files/tutorial_envfile_GFready_pooled.txt'}

In [11]:
envdfs['pooled']

,ISO,TSsd,PSsd,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,0.009381,-2.245441,1.003559,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
2,0.239071,-1.967663,0.518946,0.070353,-0.265649,0.799727,0.169512,-0.417965,0.315295
3,0.580610,-0.963343,-0.246231,0.264633,-0.248372,0.375248,-0.172690,-0.586408,-0.320803
4,0.639031,-0.164371,-0.399267,0.371616,0.167485,0.063118,-0.465978,-0.656053,-0.625240
5,-0.036985,-0.291426,-0.661614,0.244613,0.117437,-0.010939,-0.291486,-0.528748,-0.537376
...,...,...,...,...,...,...,...,...,...
96,0.113241,-0.239327,-0.054937,-0.288220,0.261489,-0.317123,-0.018548,0.228030,-0.045559
97,0.881061,0.598108,1.480884,-0.116570,0.323553,-0.231351,-0.269762,0.344912,0.059459
98,1.587251,1.669095,2.461321,0.093915,0.400397,-0.150975,-0.408936,0.588880,0.274592
99,1.381814,2.462649,2.880066,0.034177,0.399089,-0.440848,-0.464035,0.673533,0.360201


### create future climates (transplant climates) for each subpopID

In [12]:
garden_files = mvp02.create_garden_files(envdfs, envfiles)

len(garden_files['pooled'])


Creating "future climates" for each common garden ...


100

In [13]:
garden_files['pooled'][0]

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/fitting/garden_files/tutorial_envfile_GFready_pooled_1.txt'

### parallelize fit of GF models to climates of each common garden (transplant location aka future climate)

In [14]:
mvp02.run_fit_gradient_forests(garden_files)


Fitting gradient forest models to common garden climates ...

Watching 300 jobs ...


fitting gradient forests: 100%|███████████████| 300/300 [19:36<00:00,  3.92s/it]  



Redoing any jobs that died ...

Watching 34 jobs ...


iteration: 0: 100%|███████████████| 34/34 [08:56<00:00, 15.77s/it]  


In [16]:
formatclock(dt.now() - t0)

'0-00:28:38'